In [ ]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import netwulf as nw
import networkx as nx
from scipy import stats as st
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx.algorithms.community as nx_comm

In [ ]:
names = []
names_all = {}
with open('Character_names.txt', 'r') as file:
    for nick_name in file.readlines():
        names.append(nick_name[:-1])  # Ignore '\n'

In [ ]:
texts = []

In [ ]:
adjacency_matrix = np.zeros((len(names), len(names)))

# Assuming a clean text with only lower case words, no punctuation
for i, text in enumerate(texts):
    # word_array = np.array(text.split(' '))
    counts = []
    for real_name in names:
        count = 0
        for nick_name in names_all[real_name]:
            count += len(re.findall(nick_name, text))
        counts.append(count)
    # counts = np.array(counts)
    # The row is the node from which the edge comes and the column is thus the receiving node
    adjacency_matrix[i] += np.array(counts)
    # adjacency_matrix[:, i] += counts.T

In [ ]:
# Show graph before creating communities
G = nx.from_numpy_matrix(adjacency_matrix)

network, config = nw.visualize(G, config={'zoom':0.6, 'node charge':-15, 'node gravity': 0.7})
fig, ax = nw.draw_netwulf(network)
plt.savefig('StarWarsGraph')

In [ ]:
# Compute the communities
partition = nx_comm.louvain_communities(G)
# Create a list of the number of nodes in each community
sizes = [len(c) for c in partition]
print(
    f'The average community size is {round(np.mean(sizes), 2)}, with a standard deviation of {round(np.std(sizes), 2)}. The largest is of size {np.max(sizes)} and the smallest {np.min(sizes)}')

In [ ]:
# Create a np array of nodes to use for finding their positions in the graph's list
G_where = np.array(G.nodes)
# Initialize the array for colours and the values for which to colour
degreeColor = np.empty(len(G_where), dtype=object)
valueToPlot = np.log10(sizes)
# we rescale the colours to be in the RGB format (0 to 255 for three colours)
valueToPlotRescaled = 255*(valueToPlot - np.min(valueToPlot))/np.max(valueToPlot)

for size, c in zip(sizes, partition):
    for node in c:
        # Use the size to set the colour
        color = '#%02x%02x%02x' % (int(size), 0, 50)
        degreeColor[np.where(G_where == node)] = color
# zip it up into a dictionary and set it as node attribute
dictionaryColor = dict(zip(list(G.nodes), degreeColor))
nx.set_node_attributes(G, dictionaryColor, 'group')

In [ ]:
def modularity(G, communities):
    m = G.number_of_edges()
    q = .0
    for c in communities:
        lc = G.subgraph(c).number_of_edges()
        kc = sum(G.degree(n) for n in c)
        q += lc / m - (kc / (2 * m)) ** 2
    return q

modularity_from_current_communities = modularity(G, partition)